<a href="https://colab.research.google.com/github/higebobo/my-colab/blob/main/investpy_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Technical information by investing.com

## Get perfect order


## Install section

In [ ]:
!pip install investpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.3 MB/s 
     |████████████████████████████████| 235 kB 53.7 MB/s 
     |████████████████████████████████| 6.4 MB 16.9 MB/s 
  Created wheel for investpy: filename=investpy-1.0.8-py3-none-any.whl size=4481592 sha256=a2b0c7eb759df541bde93b69480ae67bc8bd5be25c6845836464dccc1cb1938b
  Stored in directory: /root/.cache/pip/wheels/96/a8/a5/0d33c72eaf00b41df7b9dc1e15d2b7c7154b3f1379ed350211
Successfully built investpy
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


## Import section

In [ ]:
import datetime

import investpy
import pandas as pd
from termcolor import colored as cl

## Get symbol list

In [ ]:
# Nikkei 225
url = 'https://site1.sbisec.co.jp/ETGate/?OutSide=on&_ControlID=WPLETmgR001Control&_PageID=WPLETmgR001Mdtl20&_DataStoreID=DSWPLETmgR001Control&_ActionID=DefaultAID&getFlg=on&burl=search_market&cat1=market&cat2=none&dir=info&file=market_meigara_225.html'
table_list = pd.read_html(url)
df_nk225_sbi = table_list[1]
df_nk225_sbi = df_nk225_sbi.iloc[:, [0, 1]]
df_nk225_sbi.columns = ['symbol', 'name']
df_index = df_nk225_sbi.set_index('symbol')

# DJI
url = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average'
table_list = pd.read_html(url)
df_dji_wp = table_list[1].iloc[:,[2,0]]
df_dji_wp.columns = ['symbol', 'name']
df_index = df_dji_wp.set_index('symbol')

## Define variables and set parameters

In [ ]:
country = 'United States'
product_type = 'stock'
interval = 'daily' # '1min', '5mins', '30mins', '1hour', '5hour', 'daily', 'weekly', 'monthly'
symbol = 'MCD'

## Technical indicators example

In [ ]:
df = investpy.technical_indicators(name=symbol, product_type=product_type,
                                   country=country, interval=interval)
if not df.empty:
    print(cl(f'Technical Indicator for {symbol} in {interval}', attrs=['bold']))
    print(df)

Technical Indicator for MCD in daily
    technical_indicator    value           signal
0               RSI(14)  57.3370              buy
1            STOCH(9,6)  69.8910              buy
2          STOCHRSI(14)  15.2110         oversold
3           MACD(12,26)   2.5200              buy
4               ADX(14)  30.3750          neutral
5           Williams %R -33.2940              buy
6               CCI(14)  40.0056          neutral
7               ATR(14)   3.5400  less_volatility
8        Highs/Lows(14)   0.0000          neutral
9   Ultimate Oscillator  58.0270              buy
10                  ROC   0.4390              buy
11  Bull/Bear Power(13)   2.0020              buy


## Total points by signal with technical indicators

In [ ]:
def get_technical_indicators(product_type, country, interval, criteria=10):
    result = []

    for symbol in df_index.index.values:
        name = df_index.loc[symbol, 'name']
        try:
            df = investpy.technical_indicators(name=symbol, product_type=product_type, country=country, interval=interval)
            result.append({'symbol': symbol, 'name': name, 'df': df})
        except:
            continue

    return result

## Show result

In [ ]:
criteria = 9
technical_indicators_list = get_technical_indicators(product_type, country, interval, criteria=criteria)

for i, x in  enumerate(technical_indicators_list):
    symbol = x['symbol']
    name = x['name']
    df = x['df']

    buy_count = (df['signal'] == 'buy').sum()
    sell_count = (df['signal'] == 'sell').sum()
    point = buy_count - sell_count
    if point > 0:
        signal = 'buy'
    elif point < 0:
        signal = 'sell'
    else:
        signal = ''
    if i == 0:
        print(cl('Signal\t\tTicker\tName', attrs=['bold']))
    if abs(point) >= criteria:
        print(f'{signal.capitalize()} ({point} pt)\t{symbol}\t{name}')

Signal		Ticker	Name
Buy (9 pt)	AAPL	Apple
Buy (9 pt)	BA	Boeing
Sell (-9 pt)	KO	Coca-Cola
Buy (9 pt)	DIS	Disney
Buy (9 pt)	HD	Home Depot
Sell (-10 pt)	JNJ	Johnson & Johnson
Buy (9 pt)	CRM	Salesforce
Sell (-10 pt)	TRV	Travelers
Buy (10 pt)	V	Visa
Buy (9 pt)	WMT	Walmart


## Filtering by specific indicators

In [ ]:
result = []
for x in technical_indicators_list:
    symbol = x['symbol']
    name = x['name']
    df = x['df']

    rsi = df.loc[0].value # RSI(14)
    adx = df.loc[4].value # ADX(14)

    if rsi >= 50 and rsi <=70 and adx >= 25:
        result.append((symbol, name, rsi, adx))

for i, x in enumerate(result):
    if i==0:
        print(cl(f'Symbol\tName{"":<16}\tRSI\tADX', attrs=['bold']))
    symbol, name, rsi, adx = x
    print(f'{symbol}\t{name:<16}\t{rsi:.1f}\t{adx:.1f}')
  

Symbol	Name                	RSI	ADX
MMM	3M              	53.7	34.6
AXP	American Express	60.3	30.8
AAPL	Apple           	62.9	41.1
BA	Boeing          	65.5	26.2
DIS	Disney          	59.2	28.5
HD	Home Depot      	69.9	58.4
HON	Honeywell       	56.4	34.4
MCD	McDonald's      	57.3	30.4
V	Visa            	61.4	37.0
WMT	Walmart         	67.9	50.1
